In [2]:
%loadFromPOM pom.xml

In [3]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import ca.uhn.fhir.validation.*;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.common.hapi.validation.validator.*;
import org.hl7.fhir.common.hapi.validation.support.*;
import ca.uhn.fhir.context.support.*;
import java.nio.file.*;


In [4]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [7]:
// read FHIR Data from C:\Project\HAPI-fhir-Lab\data\Patient-pat-example.json
String data = Files.readString(Paths.get("C:\\Project\\HAPI-fhir-Lab\\data\\Patient-pat-example.json"));

// serialize to Patient object
Patient pat = ctx.newJsonParser().parseResource(Patient.class, data);

// Create a validator and register it with the context
FhirValidator validator = ctx.newValidator();

// Create a validation module and register itwith the validator
//IValidatorModule module = new FhirInstanceValidator(ctx);

// Register the module with the validator
//validator.registerValidatorModule(module);

//NpmPackageValidationSupport npmPackageSupport = new NpmPackageValidationSupport(ctx);
//npmPackageSupport.loadPackageFromClasspath("classpath:\\data\\package.tgz");

// Create a validation support chain
ValidationSupportChain validationSupportChain = new ValidationSupportChain(
    // npmPackageSupport,
    new DefaultProfileValidationSupport(ctx),
    new InMemoryTerminologyServerValidationSupport(ctx),
    new CommonCodeSystemsTerminologyService(ctx),
    new SnapshotGeneratingValidationSupport(ctx));

// Set the chain on the validator
FhirInstanceValidator instanceValidator = new FhirInstanceValidator(validationSupportChain);
validator.registerValidatorModule(instanceValidator);

// Validate the resource
ValidationResult result = validator.validateWithResult(pat);

// Print the validation result
if (result.isSuccessful()) {
    System.out.println("Validation passed");
} else {
    System.out.println("Validation failed");
    result.getMessages().forEach(msg -> System.out.println(msg.getLocationString() + " " + msg.getMessage()));
}


Validation failed
Patient.extension[0] Unknown extension https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/person-age
Patient.identifier[0].type None of the codings provided are in the value set 'IdentifierType' (http://hl7.org/fhir/ValueSet/identifier-type), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://terminology.hl7.org/CodeSystem/v2-0203#NNxxx)
Patient.identifier[0].type.coding[0].code.extension[0] Unknown extension https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/identifier-suffix
Patient.address[0].extension[0] Unknown extension https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/tw-section
Patient.address[0].extension[1] Unknown extension https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/tw-number
Patient.address[0].extension[2] Unknown extension https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/tw-village
Patient.address[0].extension[3] Unknow